# TSNE.jl

## TSNE main code

In [150]:
using LinearAlgebra

In [151]:
N = 200
X_dim = 100
Y_dim = 2

2

In [152]:
data = randn(N,X_dim);
data_red = randn(N,Y_dim);

In [5]:
function eu_dist(data::Array{Float64,2})
    dist_mat2 = zeros(size(data)[1],size(data)[1])
    for i in 1:size(data)[1], j in i:size(data)[1]
        dist_mat2[i,j] = sum((data[i,:] - data[j,:]).^2)
        dist_mat2[j,i] = dist_mat2[i,j]
    end
    dist_mat2
end
# eu_dist(data::Matrix{Float64}) = sum(data.^2,dims=2)

eu_dist (generic function with 1 method)

In [149]:
function random_start(sample_size::Int,dim::Int;normal::Bool=false,seed::Bool=false)
    if seed
        Random.seed!(1234)
    end
    normal ? randn(sample_size,dim) : rand(sample_size,dim)
end

random_start (generic function with 1 method)

In [6]:
dist_mat = eu_dist(data);
dist_mat_red = eu_dist(data_red);

In [7]:
sum(data.^2,dims=2)

200×1 Matrix{Float64}:
 137.79881859468145
  96.68967276883663
 108.17324657544788
 102.56513013369658
  94.66035080555511
 110.52650466085
  99.65545868900443
 113.64252958860244
  76.73391567867316
  92.22826315133221
  84.75472342330943
  96.86634590770481
  88.40676071336827
   ⋮
  95.68779915122222
 100.95360915032619
  81.25687985332095
 112.9551398133758
  88.85591321198835
  84.13425480125956
 125.37604676964925
 103.82023730878183
 116.40887554463744
  87.34516189361771
 103.23732368084035
 101.9122187731591

In [8]:
function cond_gauss(data::Matrix{Float64};sigma::Float64=1.0)
    exp_mat = exp.(-eu_dist(data)/(2*sigma^2))
    for i in 1:size(exp_mat)[1]
        exp_mat[i,i] = 0.
    end
    prob = max.(exp_mat/sum(exp_mat),1e-12)
    prob, exp_mat
end

cond_gauss (generic function with 1 method)

In [9]:
function cond_t(data::Matrix{Float64};df::Int64=1)
    sum_x = sum(data.^2,dims=2)
    D = transpose(-2. * data * transpose(data) .+ sum_x) .+ sum_x
    num = 1. ./ (1. .+ D)
    for i in 1:size(num)[1]
        num[i,i] = 0.
    end
    max.(num/sum(num),1e-12), num
end

cond_t (generic function with 1 method)

In [10]:
p_distr, exp_mat = cond_gauss(data)
q_distr_g, _ = cond_gauss(data_red,sigma=1/(sqrt(2)));
q_distr_t, num = cond_t(data_red);


In [134]:
function cost_KL(p_distr::Matrix{Float64},q_distr::Matrix{Float64})
    l = log.(p_distr./q_distr)
    for i in 1:size(l)[1]
        l[i,i] = 0.
    end
    cost = sum(p_distr.*l)
end

cost_KL (generic function with 1 method)

In [12]:
function shannon_entropy(p_distr::Matrix{Float64})
    sh = p_distr.* log2.(p_distr)
    for i in 1:size(sh)[1]
        sh[i,i] = 0.
    end
    -sum(sh,dims=2)
end

perplexity(p_distr::Matrix{Float64}) = 2 .^shannon_entropy(p_distr)


perplexity (generic function with 1 method)

In [13]:
perplexity(p_distr);

In [14]:
size(data_red)

(200, 2)

In [15]:
PQ = p_distr - q_distr_t
sum(repeat(PQ[:,1] .* num[:,1],1,2) .* (transpose(data_red[1,:]) .- data_red),dims=1)

1×2 Matrix{Float64}:
 0.000415398  -0.00126871

In [79]:
function grad_KL(P::Matrix{Float64},data_red::Matrix{Float64},q_distr::Matrix{Float64},num::Matrix{Float64})
    PQ = P - q_distr
    dy = zeros(size(data_red))
    for i in 1:size(dy)[1]
        dy[i,:] = sum(repeat(PQ[:,i] .* num[:,i],1,size(data_red)[2]) .* (transpose(data_red[i,:]) .- data_red),dims=1)
    end
    dy
end

grad_KL (generic function with 2 methods)

In [17]:
dr = copy(data_red);

In [18]:
for t in 1:1000
    dy = grad_KL(data,dr)
    dr = dr - 1*dy
    if t%10 == 0
        println(t," ",cost_KL(cond_gauss(data)[1],cond_t(dr)[1]))
    end
end

10 8.0872838578418
20 8.07013807354966
30 8.053673386743656
40 8.038068342479363
50 8.023468248462633
60 8.010013529125755
70 7.997798033898676
80 7.986780418513629
90 7.976763336817375
100 7.967477431810766
110 7.958681128536075
120 7.9502050434165765
130 7.9419482135731085
140 7.933857491975354
150 7.925908093626475
160 7.918089960325183
170 7.910399333583765
180 7.902833983563147
190 7.895390837721196
200 7.888065133376637
210 7.880850467081499
220 7.873739279531051
230 7.8667234563591455
240 7.859794856332296
250 7.852945686609948
260 7.846168717847585
270 7.839457369718579
280 7.832805708825051
290 7.826208397282497
300 7.819660620598373
310 7.813158013314968
320 7.806696592719389
330 7.8002727053293945
340 7.793882987533087
350 7.78752434009277
360 7.781193915632932
370 7.774889118260109
380 7.7686076147895875
390 7.762347357473561
400 7.756106618502146
410 7.749884036779985
420 7.74367867746563
430 7.737490104374636
440 7.731318464432446
450 7.725164581731109
460 7.7190300562044

In [19]:
dr .- repeat(sum(dr,dims=1)/size(dr)[1],size(dr)[1],1)

200×2 Matrix{Float64}:
 -0.933701    2.53472
 -1.48782    -1.91453
 -0.174465    2.44526
 -0.806575   -0.256563
  0.787241    0.117058
  0.0727338  -0.117018
  0.396796   -1.33729
  2.80552    -1.72339
 -0.181783    0.509449
 -2.55173    -1.95207
  2.81935    -0.233892
  1.43268    -0.681383
 -1.51683    -2.08284
  ⋮          
  0.510146   -1.91705
  2.14844     2.01836
  0.222155   -0.801307
  0.842048    2.10632
  0.663007   -2.83952
  0.505978    0.700115
  0.618499    1.19886
 -0.752896    1.97446
 -2.04272     0.28795
 -3.431       1.51561
  0.154974   -2.3535
  1.01137     2.86957

In [137]:
function gradient_descent(T::Int64,P::Matrix{Float64},data_red::Matrix{Float64},lr::Float64;v::Bool)
    # Start iterating
    dr = copy(data_red)
    println("Beginning gradient descent...")
    for t in 1:T
        # Compute gradient
        q_distr, num = cond_t(dr)
        size(P) != size(q_distr) && throw(ArgumentError("sizes don't match"))
        dy = grad_KL(P,dr,q_distr,num)
        # Update values
        dr = dr - lr * dy
        # Compute loss
        if v && t % 10 == 0
            println("t=$t, C=$(cost_KL(P,q_distr))")
        end
        # Center in zero
        dr = dr - repeat(sum(dr,dims=1)/size(dr)[1],size(dr)[1],1)
    end
    dr
end

gradient_descent (generic function with 1 method)

In [138]:
function tsne(X::Matrix{Float64},emb_size::Int64,T::Int64;lr::Float64=1.,perp::Float64=30.,tol::Float64=1e-5,max_iter::Int=50,verbose::Bool=true)
    # Create an initial random embedding
    Y = randn(size(X)[1],emb_size)
    # Search for the sigmas of the Gaussians
    P, sigma = binary_search(X,perp,tol,max_iter,v=verbose)
    # Start the iteration
    Y_new = gradient_descent(T,P,Y,lr,v=true)
    # Y_new, P
end

tsne (generic function with 1 method)

In [ ]:
function H_sigma(D::Vector{Float64},sigma::Float64)
    P = exp.(-D * sigma) 
    sum_p = sum(P)
    H = log(sum_p) + sigma * sum(D .* P) / sum_p
    P = P / sum_p
    return H, P
end

H_sigma (generic function with 2 methods)

In [146]:
import Random 
Random.seed!(1234)
y_new, P = tsne(data,2,500,lr=0.1,verbose=false);

Beginning gradient descent...
t=10, C=1450.936404664589
t=20, C=1435.7045711204823
t=30, C=1429.1639481600748
t=40, C=1426.8987991692964
t=50, C=1426.1575636648322
t=60, C=1426.088090209072
t=70, C=1426.3245959392068
t=80, C=1426.5748731165313
t=90, C=1426.7554593261748
t=100, C=1426.891001144702
t=110, C=1427.068274092187
t=120, C=1427.3794858271517
t=130, C=1427.8257340702285
t=140, C=1428.3233048888958
t=150, C=1428.8117086510226
t=160, C=1429.2729388988837
t=170, C=1429.7047895685505
t=180, C=1430.1092903078547
t=190, C=1430.489533637054
t=200, C=1430.8485740205563
t=210, C=1431.189216901576
t=220, C=1431.5141050878715
t=230, C=1431.825691297058
t=240, C=1432.126082591617
t=250, C=1432.4169173201474
t=260, C=1432.6993478762718
t=270, C=1432.97408081828
t=280, C=1433.2414030806303
t=290, C=1433.5011706924563
t=300, C=1433.7527834624493
t=310, C=1433.9951880110102
t=320, C=1434.2269497990403
t=330, C=1434.4464175573873
t=340, C=1434.6519683635684
t=350, C=1434.8422786339834
t=360, C=

In [123]:
Q = cond_t(y_new)[1]


NaN

## Perplexity search

In [96]:
function H_sigma(D::Vector{Float64},sigma::Float64)
    P = exp.(-D * sigma)
    sum_p = sum(P)
    H = log(sum_p) + sigma * sum(D .* P) / sum_p
    P = P / sum_p
    return H, P
end

H_sigma (generic function with 2 methods)

In [116]:
function binary_search(data::Matrix{Float64},perp::Float64,tol::Float64,max_iter::Int;v::Bool)
    N, d = size(data)
    sigma = ones(N)
    P = zeros(N,N)
    sum_x = sum(data.^2,dims=2)
    D = transpose(-2. * data * transpose(data) .+ sum_x) .+ sum_x
    log_perp = log(perp)
    for i in 1:N
        if v && i % 10 == 0
            println("Perplexity search $i / $N")
        end
        sigma_low = -Inf
        sigma_high = Inf
        idx = vcat(1:i-1,i+1:N)
        # println(idx)
        D_i = D[i,idx]
        H, probs = H_sigma(D_i,sigma[i])
        delta_H = H - log_perp
        iter = 0
        while abs(delta_H) > tol && iter < max_iter
            # Perform binary update
            if delta_H > 0.
                sigma_low = sigma[i]
                (sigma_high == Inf || sigma_high == -Inf) && (sigma[i] = sigma[i] * 2.)
                (sigma_high != Inf && sigma_high != -Inf) && (sigma[i] = (sigma[i] + sigma_high) / 2.)
            else
                sigma_high = sigma[i]
                (sigma_low == Inf || sigma_low == -Inf) && (sigma[i] = sigma[i] / 2.)
                (sigma_low != Inf && sigma_low != -Inf) && (sigma[i] = (sigma[i] + sigma_low) / 2.)
            end

            H, probs = H_sigma(D_i, sigma[i])
            delta_H = H - log_perp
            iter += 1
        end
        P[i,idx] = probs
    end
    P, sigma
end

binary_search (generic function with 3 methods)

In [75]:
P = binary_search(data)[1]

Perplexity search 10 / 200
Perplexity search 20 / 200
Perplexity search 30 / 200
Perplexity search 40 / 200
Perplexity search 50 / 200
Perplexity search 60 / 200
Perplexity search 70 / 200
Perplexity search 80 / 200
Perplexity search 90 / 200
Perplexity search 100 / 200
Perplexity search 110 / 200
Perplexity search 120 / 200
Perplexity search 130 / 200
Perplexity search 140 / 200
Perplexity search 150 / 200
Perplexity search 160 / 200
Perplexity search 170 / 200
Perplexity search 180 / 200
Perplexity search 190 / 200
Perplexity search 200 / 200


200×200 Matrix{Float64}:
 0.0          0.013009     0.00174562   …  0.000453386  0.000478851
 0.000143973  0.0          6.62514e-6      2.48162e-5   9.84478e-6
 9.0575e-5    3.79863e-5   0.0             0.000275414  0.000690368
 0.000340016  0.000835163  0.00121974      0.000996583  0.000530297
 1.9457e-5    0.000572194  0.0141666       1.03513e-5   2.84998e-5
 3.50639e-6   0.000248716  3.00713e-5   …  0.00091807   3.67982e-6
 4.125e-7     0.00649562   1.07812e-5      4.54708e-5   8.96195e-5
 6.53741e-5   0.0342169    1.27029e-5      0.000118655  0.0123752
 3.83155e-7   0.000314369  0.00842069      8.69832e-5   0.000339723
 6.51878e-6   0.000399306  0.00115832      0.00221378   0.00259398
 1.42411e-5   0.000563238  0.000275948  …  0.000535009  0.00194153
 1.36409e-7   0.000128826  0.00196263      0.00215751   0.00130628
 0.00016258   0.000648007  1.45371e-5      0.00258399   0.000165553
 ⋮                                      ⋱               
 6.12407e-7   0.000331047  9.35834e-5      

## PCA

In [197]:
using Statistics
function PCA(data::Matrix{Float64},dims::Int)
    N, d = size(data)
    data_std = data - repeat(mean(data,dims=1),N,1)
    e, v = eigen(transpose(data_std) * data_std);
    v = v[:,sortperm(e,rev=true)]
    Y = data_std * v[:,1:dims]
end
PCA(data,50)

200×50 Matrix{Float64}:
  1.41159     1.13304   -2.32955   …   1.2008      1.12731   -0.124733
 -1.40426     0.365179  -2.04318       0.128689   -0.633356  -0.0650762
  0.0131935   2.05322   -0.424915     -0.0738174  -0.346675  -0.730506
  2.05734     3.75658   -0.738659      0.860738   -0.51819    1.3197
 -1.65809     0.194754  -0.156952      0.598442   -1.11275    0.212128
  2.78893    -1.66635    2.46155   …   2.08899    -1.04129    1.20618
  0.0662939  -1.8884     1.32013      -0.517124    0.263401   0.708997
  1.82423    -1.05689    3.4902        0.870149   -1.05058   -0.117922
 -0.895337   -0.9998    -2.43967       0.427194    0.23457   -0.120136
  1.12525    -0.234484   1.08868       0.79763     0.53812   -1.09673
  1.57057     0.355837   2.79963   …   0.701429   -1.89189    0.906449
  0.596955   -0.990265   0.844846      0.551239   -0.743307   0.486164
  2.03832     0.293981   0.421912      0.590694    0.642188   1.74711
  ⋮                                ⋱                     

In [179]:
?eigvecs

search: eigvecs



```
eigvecs(A::SymTridiagonal[, eigvals]) -> Matrix
```

Return a matrix `M` whose columns are the eigenvectors of `A`. (The `k`th eigenvector can be obtained from the slice `M[:, k]`.)

If the optional vector of eigenvalues `eigvals` is specified, `eigvecs` returns the specific corresponding eigenvectors.

# Examples

```jldoctest
julia> A = SymTridiagonal([1.; 2.; 1.], [2.; 3.])
3×3 SymTridiagonal{Float64, Vector{Float64}}:
 1.0  2.0   ⋅
 2.0  2.0  3.0
  ⋅   3.0  1.0

julia> eigvals(A)
3-element Vector{Float64}:
 -2.1400549446402604
  1.0000000000000002
  5.140054944640259

julia> eigvecs(A)
3×3 Matrix{Float64}:
  0.418304  -0.83205      0.364299
 -0.656749  -7.39009e-16  0.754109
  0.627457   0.5547       0.546448

julia> eigvecs(A, [1.])
3×1 Matrix{Float64}:
  0.8320502943378438
  4.263514128092366e-17
 -0.5547001962252291
```

---

```
eigvecs(A; permute::Bool=true, scale::Bool=true, `sortby`) -> Matrix
```

Return a matrix `M` whose columns are the eigenvectors of `A`. (The `k`th eigenvector can be obtained from the slice `M[:, k]`.) The `permute`, `scale`, and `sortby` keywords are the same as for [`eigen`](@ref).

# Examples

```jldoctest
julia> eigvecs([1.0 0.0 0.0; 0.0 3.0 0.0; 0.0 0.0 18.0])
3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0
```

---

```
eigvecs(A, B) -> Matrix
```

Return a matrix `M` whose columns are the generalized eigenvectors of `A` and `B`. (The `k`th eigenvector can be obtained from the slice `M[:, k]`.)

# Examples

```jldoctest
julia> A = [1 0; 0 -1]
2×2 Matrix{Int64}:
 1   0
 0  -1

julia> B = [0 1; 1 0]
2×2 Matrix{Int64}:
 0  1
 1  0

julia> eigvecs(A, B)
2×2 Matrix{ComplexF64}:
  0.0+1.0im   0.0-1.0im
 -1.0+0.0im  -1.0-0.0im
```
